## Hierarchical Models

## Endponits:
 - (1) EPA category (class 1-4), 
 - (2) GHS category (class 1-5), 
 - (3) LD50 (mmol/kg); 
 - (4) toxic (LD50 < 2,000 mg/kg == 1)
 - (5) very toxic (LD50 < 50 mg/kg == 1)
 
 
## Algorithms:
 - Random Forest
 - SVM & SVR
 - XGBOOST
 - kNN
 
All model were tuned with 5-fold cross-validation.

## Endpoint 1: Toxic
 - kNN: {'n_neighbors': 71, 'p': 1, 'weights': 'distance'}; Best score: 0.880
 - SVM: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}; Best score: 0.883
 - RF: {'n_estimators': 1500, 'min_samples_split': 5, 'min_samples_leaf': 6, 'max_features': 'log2', 'max_depth': 35, 'bootstrap': False}; Best score: 0.882
 - xgb: {'subsample': 0.6, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.5}; Best score: 0.883
 
## Endpoint 2: LD50
 - RF: {'n_estimators': 1500, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}; Best score: 0.304
 - xgb: {'subsample': 0.6, 'n_estimators': 500, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.01, 'gamma': 1, 'colsample_bytree': 0.6}; Best score: 0.304
 - svm: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}; Best score: 0.304
 - kNN: {'n_neighbors': 45, 'p': 2, 'weights': 'distance'}; Best score: 0.315
 
## Endpoint 3: Multiclass
 - RF: {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}; Best score: 0.641
 - xgb: {'subsample': 0.6, 'n_estimators': 1500, 'min_child_weight': 3, 'max_depth': 10, 'learning_rate': 0.01, 'gamma': 5, 'colsample_bytree': 0.7}; Best score: 0.640
 - svm: {'C': 0.1, 'kernel': 'linear'} Best score: 0.636
 - knn: {'n_neighbors': 35, 'p': 1, 'weights': 'distance'}; Best score: 0.631

In [1]:
from utils import * 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import itertools
from pprint import pprint
import joblib

import statistics

# Models
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import KFold, cross_validate, GridSearchCV, cross_val_score, RandomizedSearchCV 
from sklearn.model_selection import cross_val_predict

from sklearn.pipeline import Pipeline

from sklearn.metrics import make_scorer

#regression matrics
from sklearn.metrics import mean_absolute_error , mean_squared_error, r2_score

#classification metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score, matthews_corrcoef

from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.base import TransformerMixin
from sklearn.base import clone
from sklearn.model_selection._split import check_cv

In [3]:
train_labels = pd.read_csv('../data/train_test_sets/train_labels.csv', index_col = 'CASRN')
train_labels.shape

(8221, 6)

In [4]:
train_Hfeatures = pd.read_csv('../data/Hmodel_features_combined/train_Hfeatures.csv', index_col = 'CASRN')
train_Hfeatures.shape

(8221, 100)

## Endpoint 1: Toxic

In [7]:
%%time

endpoint = 'Toxic'
descriptor = 'Hmodel'
algorithm = 'RF'
name = f'{endpoint}_{algorithm}_{descriptor}'

encoder_toxic = joblib.load('../data/label_encoders/encoder_toxic.joblib')

# model
rf_clf = RandomForestClassifier(random_state =42, n_jobs=6,
                              n_estimators = 1500, min_samples_split = 5, min_samples_leaf=6,
                              max_features = 'log2', max_depth=35, bootstrap= False)

#input
a, b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'toxic', encoder = encoder_toxic)

# results
BCM_mf,  BCM_oof, BCM_base_model, cv_score  = Classification_meta_features(rf_clf, a, c, b, d, e,cv=10,n_jobs=1, 
                                                      col_names = [f'{name}-0', f'{name}-1'])
# report the results
report_clf_models(cv_score)

# Save results
# BCM_mf.to_csv(f'../data/Hmodel_features/{name}.csv') # no need for this 
np.save(f'../results/Hierarchical_models/{name}.npy', BCM_oof)
joblib.dump(BCM_base_model, f'../models/Hierarchical_models/{name}.pkl')
joblib.dump(cv_score, f'../results/Hierarchical_models/{name}_CVScore')


Accuracy: 0.805 std: 0.015
Balance Accuracy: 0.799 std: 0.014
matthews_corrcoef: 0.601 std: 0.029
f1_score: 0.804 std: 0.015
AUROC: 0.799 std: 0.014
CPU times: user 2min 59s, sys: 12.2 s, total: 3min 11s
Wall time: 5min 33s


In [11]:
%%time

endpoint = 'Toxic'
descriptor = 'Hmodel'
algorithm = 'SVM'
name = f'{endpoint}_{algorithm}_{descriptor}'

encoder_toxic = joblib.load('../data/label_encoders/encoder_toxic.joblib')

# model
clf = SVC(random_state=42, probability=True,
          C = 10, gamma = 0.001, kernel = 'rbf')

#input
a, b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'toxic', encoder = encoder_toxic)

# results
BCM_mf,  BCM_oof, BCM_model, cv_score  = Classification_meta_features(clf, a, c, b, d, e,cv=10,n_jobs=6, 
                                                      col_names = [f'{name}-0', f'{name}-1'])
# report the results
report_clf_models(cv_score)

# Save results
# BCM_mf.to_csv(f'../data/Hmodel_features/{name}.csv') # no need for this 
np.save(f'../results/Hierarchical_models/{name}.npy', BCM_oof)
joblib.dump(BCM_model, f'../models/Hierarchical_models/{name}.pkl')
joblib.dump(cv_score, f'../results/Hierarchical_models/{name}_CVScore')


Accuracy: 0.799 std: 0.015
Balance Accuracy: 0.792 std: 0.015
matthews_corrcoef: 0.588 std: 0.03
f1_score: 0.798 std: 0.015
AUROC: 0.792 std: 0.015
CPU times: user 25 s, sys: 308 ms, total: 25.3 s
Wall time: 3min 26s


In [12]:
%%time

endpoint = 'Toxic'
descriptor = 'Hmodel'
algorithm = 'xgboost'
name = f'{endpoint}_{algorithm}_{descriptor}'

encoder_toxic = joblib.load('../data/label_encoders/encoder_toxic.joblib')

# model
clf = XGBClassifier(random_state =123, n_jobs=6,
                    subsample = 0.6, n_estimators = 500, min_child_weight=1,
                    max_depth = 3, learning_rate=0.01, gamma= 0,
                    colsample_bytree = 0.5)


#input
a, b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'toxic', encoder = encoder_toxic)

# results
BCM_mf,  BCM_oof, BCM_model, cv_score  = Classification_meta_features(clf, a, c, b, d, e,cv=10,n_jobs=1, 
                                                      col_names = [f'{name}-0', f'{name}-1'])
# report the results
report_clf_models(cv_score)

# Save results
# BCM_mf.to_csv(f'../data/Hmodel_features/{name}.csv') # no need for this 
np.save(f'../results/Hierarchical_models/{name}.npy', BCM_oof)
joblib.dump(BCM_model, f'../models/Hierarchical_models/{name}.pkl')
joblib.dump(cv_score, f'../results/Hierarchical_models/{name}_CVScore')


Accuracy: 0.802 std: 0.011
Balance Accuracy: 0.795 std: 0.01
matthews_corrcoef: 0.594 std: 0.021
f1_score: 0.801 std: 0.011
AUROC: 0.795 std: 0.01
CPU times: user 4min 56s, sys: 375 ms, total: 4min 56s
Wall time: 50.2 s


In [14]:
%%time

endpoint = 'Toxic'
descriptor = 'Hmodel'
algorithm = 'knn'
name = f'{endpoint}_{algorithm}_{descriptor}'

encoder_toxic = joblib.load('../data/label_encoders/encoder_toxic.joblib')

# model
clf = KNeighborsClassifier(n_neighbors = 71, weights = 'distance')

#input
a, b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'toxic', encoder = encoder_toxic)

# results
BCM_mf,  BCM_oof, BCM_model, cv_score  = Classification_meta_features(clf, a, c, b, d, e,cv=10,n_jobs=6, 
                                                      col_names = [f'{name}-0', f'{name}-1'])
# report the results
report_clf_models(cv_score)

# Save results
# BCM_mf.to_csv(f'../data/Hmodel_features/{name}.csv') # no need for this 
np.save(f'../results/Hierarchical_models/{name}.npy', BCM_oof)
joblib.dump(BCM_model, f'../models/Hierarchical_models/{name}.pkl')
joblib.dump(cv_score, f'../results/Hierarchical_models/{name}_CVScore')


Accuracy: 0.796 std: 0.017
Balance Accuracy: 0.79 std: 0.017
matthews_corrcoef: 0.584 std: 0.034
f1_score: 0.796 std: 0.017
AUROC: 0.79 std: 0.017
CPU times: user 140 ms, sys: 52.4 ms, total: 192 ms
Wall time: 18.7 s


## Endpoint 2: EPA

In [15]:
%%time

endpoint = 'EPA'
descriptor = 'Hmodel'
algorithm = 'RF'
name = f'{endpoint}_{algorithm}_{descriptor}'

encoder_epa = joblib.load('../data/label_encoders/encoder_epa.joblib')

# model
clf = RandomForestClassifier(random_state =42, n_jobs=6,
                              n_estimators = 500, min_samples_split = 10, min_samples_leaf=6,
                              max_features = 'sqrt', max_depth=None, bootstrap= False)

#input
a, b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'EPA_category', encoder = encoder_epa)

# results
MCM_mf,  MCM_oof, MCM_model, cv_score  = Classification_meta_features(clf, a, c, b, d, e,cv=10,n_jobs=1, 
                                                      col_names = [f'{name}-1', f'{name}-2', f'{name}-3', f'{name}-4'])
# report the results
report_clf_models(cv_score)

# Save results
np.save(f'../results/Hierarchical_models/{name}.npy', MCM_oof)
joblib.dump(MCM_model, f'../models/Hierarchical_models/{name}.pkl')
joblib.dump(cv_score, f'../results/Hierarchical_models/{name}_CVScore')


Accuracy: 0.659 std: 0.016
Balance Accuracy: 0.587 std: 0.015
matthews_corrcoef: 0.458 std: 0.023
f1_score: 0.648 std: 0.017
AUROC: 0.707 std: 0.012
CPU times: user 1min 27s, sys: 4.99 s, total: 1min 32s
Wall time: 3min 8s


In [16]:
%%time

endpoint = 'EPA'
descriptor = 'Hmodel'
algorithm = 'xgboost'
name = f'{endpoint}_{algorithm}_{descriptor}'

encoder_epa = joblib.load('../data/label_encoders/encoder_epa.joblib')

# model
clf = XGBClassifier(random_state =123, n_jobs=6,
                    subsample = 0.6, n_estimators = 1500, min_child_weight=3,
                    max_depth = 10, learning_rate=0.01, gamma= 5,
                    colsample_bytree = 0.7)

#input
a, b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'EPA_category', encoder = encoder_epa)

# results
MCM_mf,  MCM_oof, MCM_model, cv_score  = Classification_meta_features(clf, a, c, b, d, e,cv=10,n_jobs=1, 
                                                      col_names = [f'{name}-1', f'{name}-2', f'{name}-3', f'{name}-4'])
# report the results
report_clf_models(cv_score)

# Save results
np.save(f'../results/Hierarchical_models/{name}.npy', MCM_oof)
joblib.dump(MCM_model, f'../models/Hierarchical_models/{name}.pkl')
joblib.dump(cv_score, f'../results/Hierarchical_models/{name}_CVScore')


Accuracy: 0.658 std: 0.015
Balance Accuracy: 0.588 std: 0.012
matthews_corrcoef: 0.456 std: 0.023
f1_score: 0.647 std: 0.017
AUROC: 0.708 std: 0.013
CPU times: user 3h 33min 48s, sys: 16.1 s, total: 3h 34min 4s
Wall time: 36min 12s


In [17]:
%%time

endpoint = 'EPA'
descriptor = 'Hmodel'
algorithm = 'knn'
name = f'{endpoint}_{algorithm}_{descriptor}'

encoder_epa = joblib.load('../data/label_encoders/encoder_epa.joblib')

# model
clf = KNeighborsClassifier(n_neighbors = 35, weights = 'distance', p=1)

#input
a, b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'EPA_category', encoder = encoder_epa)

# results
MCM_mf,  MCM_oof, MCM_model, cv_score  = Classification_meta_features(clf, a, c, b, d, e,cv=10,n_jobs=6, 
                                                      col_names = [f'{name}-1', f'{name}-2', f'{name}-3', f'{name}-4'])
# report the results
report_clf_models(cv_score)

# Save results
np.save(f'../results/Hierarchical_models/{name}.npy', MCM_oof)
joblib.dump(MCM_model, f'../models/Hierarchical_models/{name}.pkl')
joblib.dump(cv_score, f'../results/Hierarchical_models/{name}_CVScore')


Accuracy: 0.654 std: 0.018
Balance Accuracy: 0.576 std: 0.013
matthews_corrcoef: 0.447 std: 0.023
f1_score: 0.64 std: 0.02
AUROC: 0.7 std: 0.014
CPU times: user 239 ms, sys: 172 ms, total: 411 ms
Wall time: 24.1 s


In [18]:
%%time

endpoint = 'EPA'
descriptor = 'Hmodel'
algorithm = 'SVM'
name = f'{endpoint}_{algorithm}_{descriptor}'

encoder_epa = joblib.load('../data/label_encoders/encoder_epa.joblib')

# model
clf = SVC(random_state=42, probability=True,
          C = 0.1, kernel = 'linear')

#input
a, b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'EPA_category', encoder = encoder_epa)

# results
MCM_mf,  MCM_oof, MCM_model, cv_score  = Classification_meta_features(clf, a, c, b, d, e,cv=10,n_jobs=6, 
                                                      col_names = [f'{name}-1', f'{name}-2', f'{name}-3', f'{name}-4'])
# report the results
report_clf_models(cv_score)

# Save results
np.save(f'../results/Hierarchical_models/{name}.npy', MCM_oof)
joblib.dump(MCM_model, f'../models/Hierarchical_models/{name}.pkl')
joblib.dump(cv_score, f'../results/Hierarchical_models/{name}_CVScore')


/home/xinhao/.local/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Accuracy: 0.655 std: 0.015
Balance Accuracy: 0.569 std: 0.013
matthews_corrcoef: 0.448 std: 0.018
f1_score: 0.639 std: 0.017
AUROC: 0.699 std: 0.01
CPU times: user 25.4 s, sys: 228 ms, total: 25.6 s
Wall time: 3min 50s


## Endpoint 3: LD50

In [19]:
%%time

endpoint = 'LD50'
descriptor = 'Hmodel'
algorithm = 'RF'
name = f'{endpoint}_{algorithm}_{descriptor}'

# model
rf_reg = RandomForestRegressor(random_state =42, n_jobs=6,
                              n_estimators = 1500, min_samples_split = 2, min_samples_leaf=4,
                              max_features = 'sqrt', max_depth=20, bootstrap= False)

#input
a, b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'logLD50_mmolkg')

# results
RM_mf, RM_oof, RM_model, cv_score = Regression_meta_features(rf_reg, a, c, b, 
                                                       d, e,cv=10, n_jobs = 1, col_names = [f'{name}'])
# report the results
report_cv_reg_models(cv_score)

# Save results
np.save(f'../results/Hierarchical_models/{name}.npy', RM_oof)
joblib.dump(RM_model, f'../models/Hierarchical_models/{name}.pkl')
joblib.dump(cv_score, f'../results/Hierarchical_models/{name}_CVScore')


RMSE: 0.549 std: 0.025
R2: 0.631 std: 0.028
MAE: 0.396 std: 0.019
MSE: 0.301 std: 0.027
CPU times: user 2min 55s, sys: 6.4 s, total: 3min 1s
Wall time: 6min 51s


In [21]:
%%time

endpoint = 'LD50'
descriptor = 'Hmodel'
algorithm = 'xgboost'
name = f'{endpoint}_{algorithm}_{descriptor}'

# model
reg = XGBRegressor(random_state =123, n_jobs=6, objective ='reg:squarederror',
                    subsample = 0.6, n_estimators = 500, min_child_weight=3,
                    max_depth = 6, learning_rate=0.01, gamma= 1,
                    colsample_bytree = 0.6)

#input
a, b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'logLD50_mmolkg')

# results
RM_mf, RM_oof, RM_model, cv_score = Regression_meta_features(reg, a, c, b, 
                                                       d, e,cv=10, n_jobs = 1, col_names = [f'{name}'])
# report the results
report_cv_reg_models(cv_score)

# Save results
np.save(f'../results/Hierarchical_models/{name}.npy', RM_oof)
joblib.dump(RM_model, f'../models/Hierarchical_models/{name}.pkl')
joblib.dump(cv_score, f'../results/Hierarchical_models/{name}_CVScore')


RMSE: 0.55 std: 0.024
R2: 0.628 std: 0.027
MAE: 0.398 std: 0.018
MSE: 0.303 std: 0.027
CPU times: user 9min 14s, sys: 982 ms, total: 9min 15s
Wall time: 1min 33s


In [22]:
%%time

endpoint = 'LD50'
descriptor = 'Hmodel'
algorithm = 'knn'
name = f'{endpoint}_{algorithm}_{descriptor}'

# model
reg = KNeighborsRegressor(p = 2, n_neighbors = 45, weights = 'distance')

#input
a, b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'logLD50_mmolkg')

# results
RM_mf, RM_oof, RM_model, cv_score = Regression_meta_features(reg, a, c, b, 
                                                       d, e,cv=10, n_jobs = 6, col_names = [f'{name}'])
# report the results
report_cv_reg_models(cv_score)

# Save results
np.save(f'../results/Hierarchical_models/{name}.npy', RM_oof)
joblib.dump(RM_model, f'../models/Hierarchical_models/{name}.pkl')
joblib.dump(cv_score, f'../results/Hierarchical_models/{name}_CVScore')


RMSE: 0.559 std: 0.024
R2: 0.617 std: 0.028
MAE: 0.405 std: 0.018
MSE: 0.312 std: 0.027
CPU times: user 1.54 s, sys: 176 ms, total: 1.72 s
Wall time: 15.8 s


In [23]:
%%time

endpoint = 'LD50'
descriptor = 'Hmodel'
algorithm = 'SVM'
name = f'{endpoint}_{algorithm}_{descriptor}'

# model
reg = SVR(C = 1, gamma = 0.01, kernel = 'rbf')

#input
a, b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'logLD50_mmolkg')

# results
RM_mf, RM_oof, RM_model, cv_score = Regression_meta_features(reg, a, c, b, 
                                                       d, e,cv=10, n_jobs = 6, col_names = [f'{name}'])
# report the results
report_cv_reg_models(cv_score)

# Save results
np.save(f'../results/Hierarchical_models/{name}.npy', RM_oof)
joblib.dump(RM_model, f'../models/Hierarchical_models/{name}.pkl')
joblib.dump(cv_score, f'../results/Hierarchical_models/{name}_CVScore')


RMSE: 0.551 std: 0.024
R2: 0.628 std: 0.026
MAE: 0.395 std: 0.018
MSE: 0.304 std: 0.026
CPU times: user 5.29 s, sys: 64 ms, total: 5.35 s
Wall time: 29.2 s


## Get the predictrions on test set

In [24]:
test_Hfeatures = pd.read_csv('../data/Hmodel_features_combined/test_Hfeatures.csv', index_col = 'CASRN')
test_Hfeatures.shape

(2849, 100)

In [26]:
%%time

index = test_Hfeatures.index

model_path = '../models/Hierarchical_models/'
result_path = '../results/Hierarchical_testset_preds/'

endpoints = ['Toxic', 'EPA', 'LD50']
descriptors = ['Hmodel']
algorithms = ['RF', 'SVM', 'knn', 'xgboost']

feature = test_Hfeatures.values.astype('float32')

for e in endpoints:
    for d in descriptors:
        for a in algorithms:
            name = f'{e}_{a}_{d}'
            print(f'{name}: computing....')
            model = joblib.load(f'{model_path}{name}.pkl')
            
            if e == 'Toxic':
                predictions = model.predict_proba(feature)
                df = pd.DataFrame(predictions, columns=[f'{name}-0', f'{name}-1'],index = index)
                df.to_csv(f'{result_path}{name}.csv')

                print(f'{name}: saved')
            if e == 'EPA':
                predictions = model.predict_proba(feature)
                df = pd.DataFrame(predictions, columns=[f'{name}-1', f'{name}-2', f'{name}-3', f'{name}-4'], index = index)
                df.to_csv(f'{result_path}{name}.csv')

                print(f'{name}: saved')
            if e == 'LD50':
                predictions = model.predict(feature)
                df = pd.DataFrame(predictions, columns=[f'{name}'],index = index)
                df.to_csv(f'{result_path}{name}.csv')
                print(f'{name}: saved') 

Toxic_RF_Hmodel: computing....
Toxic_RF_Hmodel: saved
Toxic_SVM_Hmodel: computing....
Toxic_SVM_Hmodel: saved
Toxic_knn_Hmodel: computing....
Toxic_knn_Hmodel: saved
Toxic_xgboost_Hmodel: computing....
Toxic_xgboost_Hmodel: saved
EPA_RF_Hmodel: computing....
EPA_RF_Hmodel: saved
EPA_SVM_Hmodel: computing....
EPA_SVM_Hmodel: saved
EPA_knn_Hmodel: computing....
EPA_knn_Hmodel: saved
EPA_xgboost_Hmodel: computing....
EPA_xgboost_Hmodel: saved
LD50_RF_Hmodel: computing....
LD50_RF_Hmodel: saved
LD50_SVM_Hmodel: computing....
LD50_SVM_Hmodel: saved
LD50_knn_Hmodel: computing....
LD50_knn_Hmodel: saved
LD50_xgboost_Hmodel: computing....
LD50_xgboost_Hmodel: saved
CPU times: user 18.9 s, sys: 560 ms, total: 19.4 s
Wall time: 15.5 s
